## Trying out a linear model: 

Based on: https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models/code
Author: Alexandru Papiu    
If you use parts of this notebook in your own scripts, please give some sort of credit (for example link back to this). Thanks!


There have been a few [great](https://www.kaggle.com/comartel/house-prices-advanced-regression-techniques/house-price-xgboost-starter/run/348739)  [scripts](https://www.kaggle.com/zoupet/house-prices-advanced-regression-techniques/xgboost-10-kfolds-with-scikit-learn/run/357561) on [xgboost](https://www.kaggle.com/tadepalli/house-prices-advanced-regression-techniques/xgboost-with-n-trees-autostop-0-12638/run/353049) already so I'd figured I'd try something simpler: a regularized linear regression model. Surprisingly it does really well with very little feature engineering. The key point is to to log_transform the numeric variables since most of them are skewed.

In [ ]:
import pandas as pd # for data storing
import numpy as np # numeric staff
import seaborn as sns # making plots beautifull
import matplotlib # for plotting

import matplotlib.pyplot as plt # for plotting
from scipy.stats import skew
from scipy.stats.stats import pearsonr


%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

Get data at: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:
numeric_features = all_data.dtypes[all_data.dtypes != "object"].index

In [ ]:
all_data = pd.get_dummies(all_data)

In [ ]:
all_data

In [ ]:
all_data = all_data.fillna(all_data.mean())

In [ ]:
from sklearn.preprocessing import scale
from sklearn.utils import shuffle
all_data_scaled = pd.DataFrame(scale(all_data), index = all_data.index, columns=all_data.columns)

In [ ]:
#creating matrices for sklearn:
X_train = all_data_scaled[:train.shape[0]]

X_test = all_data_scaled[train.shape[0]:]
y = np.log1p(train.SalePrice)

X_train, y = shuffle(X_train, y)

In [ ]:
all_data

In [ ]:
all_data.shape

### Models

Now we are going to use regularized linear regression models from the scikit learn module. I'm going to try both l_1(Lasso) and l_2(Ridge) regularization. I'll also define a function that returns the cross-validation rmse error so we can evaluate our models and pick the best tuning par

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, Lasso
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import get_scorer 

def rmse_cv(model, data = X_train):
    train_rmse = []
    test_rmse = []
    scorer = get_scorer("neg_mean_squared_error")
    for train_index, test_index in KFold(len(y), n_folds=5):
        model.fit(data.loc[train_index], y[train_index])
        train_rmse.append(np.sqrt(-scorer(model, data.loc[train_index], y[train_index])))
        test_rmse.append(np.sqrt(-scorer(model, data.loc[test_index], y[test_index])))
    return(np.array(train_rmse), np.array(test_rmse))

## Ridge

In [ ]:
model_ridge = Ridge()

The main tuning parameter for the Ridge model is alpha - a regularization parameter that measures how flexible our model is. The higher the regularization the less prone our model will be to overfit. However it will also lose flexibility and might not capture all of the signal in the data.

In [ ]:
model_ridge.fit(X_train, y)

plt.figure(figsize=(8,6))
plt.plot(y, model_ridge.predict(X_train), 'o')
plt.xlabel("true log(price)", fontsize=16)
plt.ylabel("predicted log(price)", fontsize=16)

In [ ]:
ridge_alphas = [0.01, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75, 100, 125,
              150, 200, 225, 250, 275, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300,1400, 1500]
train_cv_ridge, test_cv_ridge = zip(*[rmse_cv(Ridge(alpha = alpha))
            for alpha in ridge_alphas])
cv_ridge_mean = np.array(map(lambda x:x.mean(), test_cv_ridge))
cv_ridge_std = np.array(map(lambda x:x.std(), test_cv_ridge))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(ridge_alphas, cv_ridge_mean)
#plt.fill_between(ridge_alphas, cv_ridge_mean-cv_ridge_std, cv_ridge_mean+cv_ridge_std, alpha=0.4)

plt.plot(ridge_alphas, map(lambda x:x.mean(), train_cv_ridge))
plt.legend(['test_score', 'train_score'])
plt.xlabel("alpha", fontsize=16)
plt.ylabel("rmse", fontsize=16)

Note the U-ish shaped curve above. When alpha is too large the regularization is too strong and the model cannot capture all the complexities in the data. If however we let the model be too flexible (alpha small) the model begins to overfit. A value of alpha = 10 is about right based on the plot above.

In [ ]:
cv_ridge_mean.min()

## LASSO

In [ ]:
lasso_alphas = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.02]
train_cv_lasso, test_cv_lasso = zip(*[rmse_cv(Lasso(alpha = alpha))
                                     for alpha in lasso_alphas])
cv_lasso_mean = np.array(map(lambda x:x.mean(), test_cv_lasso))
cv_lasso_std = np.array(map(lambda x:x.std(), test_cv_lasso))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(lasso_alphas, cv_lasso_mean)
plt.fill_between(lasso_alphas, cv_lasso_mean-cv_lasso_std, cv_lasso_mean+cv_lasso_std, alpha=0.4)

plt.plot(lasso_alphas, map(lambda x:x.mean(), train_cv_lasso))
plt.legend(['test_rmse', 'train_rmse'])
plt.xlabel("alpha", fontsize=16)
plt.ylabel("rmse", fontsize=16)

So for the Ridge regression we get a rmsle of about 0.127

Let' try out the Lasso model. We will do a slightly different approach here and use the built in Lasso CV to figure out the best alpha for us. For some reason the alphas in Lasso CV are really the inverse or the alphas in Ridge.

In [ ]:
model_lasso = Lasso(alpha = 0.005).fit(X_train, y)

In [ ]:
rmse_cv(model_lasso)[1].mean(), rmse_cv(model_lasso)[1].std()

In [ ]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

Good job Lasso.  One thing to note here however is that the features selected are not necessarily the "correct" ones - especially since there are a lot of collinear features in this dataset. One idea to try here is run Lasso a few times on boostrapped samples and see how stable the feature selection is.

We can also take a look directly at what the most important coefficients are:

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Lasso Model")

The most important positive feature is `GrLivArea` -  the above ground area by area square feet. This definitely sense. Then a few other  location and quality features contributed positively. Some of the negative features make less sense and would be worth looking into more - it seems like they might come from unbalanced categorical variables.

 Also note that unlike the feature importance you'd get from a random forest these are _actual_ coefficients in your model - so you can say precisely why the predicted price is what it is. The only issue here is that we log_transformed both the target and the numeric features so the actual magnitudes are a bit hard to interpret. 

The residual plot looks pretty good.To wrap it up let's predict on the test set and submit on the leaderboard:

In [ ]:
def add_modifications_to_numeric(df):
    for feature in numeric_features:
        #print df[feature].min()
        #df[feature+'_log1plog1p'] = np.log1p(np.log1p(df[feature]-df[feature].min()))
        df[feature+'_log1p'] = np.log1p(df[feature]-df[feature].min())
        #df[feature+'_sqrt'] = np.sqrt(df[feature]-df[feature].min())
    return df

In [ ]:
all_data_modified = pd.DataFrame(scale(add_modifications_to_numeric(all_data).as_matrix()),
                                 index = all_data.index, columns=all_data.columns)

X_train_new = all_data_modified[:train.shape[0]]
X_test_new = all_data_modified[train.shape[0]:]
del all_data_modified # to save RAM

In [ ]:
lasso_alphas = [0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.02]
train_cv_lasso, test_cv_lasso = zip(*[rmse_cv(Lasso(alpha = alpha), data=X_train_new)
                                    for alpha in lasso_alphas])
cv_lasso_mean = np.array(map(lambda x:x.mean(), test_cv_lasso))
cv_lasso_std = np.array(map(lambda x:x.std(), test_cv_lasso))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(lasso_alphas, cv_lasso_mean)
plt.fill_between(lasso_alphas, cv_lasso_mean-cv_lasso_std, cv_lasso_mean+cv_lasso_std, alpha=0.4)
plt.xlabel("alpha", fontsize=16)
plt.ylabel("rmse", fontsize=16)

In [ ]:
model_lasso = Lasso(alpha = 0.005).fit(X_train_new, y)

In [ ]:
coef = pd.Series(model_lasso.coef_, index = X_train_new.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

## Random Forest

Let's add an xgboost model to our linear model to see if we can improve our score:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
max_depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
train_cv_rf, test_cv_rf = zip(*[rmse_cv(RandomForestRegressor(n_estimators=100, max_depth=depth))
                                     for depth in max_depths])
cv_rf_mean = np.array(map(lambda x:x.mean(), test_cv_rf))
cv_rf_std = np.array(map(lambda x:x.std(), test_cv_rf))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(max_depths, cv_rf_mean)
plt.fill_between(max_depths, cv_rf_mean-cv_rf_std, cv_rf_mean+cv_rf_std, alpha=0.4)
plt.plot(max_depths, map(lambda x:x.mean(), train_cv_rf))
plt.xlabel("depth", fontsize=16)
plt.ylabel("rmse", fontsize=16)

In [ ]:
max_features_list = range(10,200,3)
train_cv_rf, test_cv_rf = zip(*[rmse_cv(RandomForestRegressor(n_estimators=100,
                                                              max_depth=12,
                                                              min_samples_split = 2,
                                                              max_features = max_features, n_jobs=3))
                                     for max_features in max_features_list])
cv_rf_mean = np.array(map(lambda x:x.mean(), test_cv_rf))
cv_rf_std = np.array(map(lambda x:x.std(), test_cv_rf))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(max_features_list, cv_rf_mean)
plt.fill_between(max_features_list, cv_rf_mean-cv_rf_std, cv_rf_mean+cv_rf_std, alpha=0.4)
plt.plot(max_features_list, map(lambda x:x.mean(), train_cv_rf))
plt.xlabel("", fontsize=16)
plt.ylabel("rmse", fontsize=16)

In [ ]:
RandomForestRegressor()

In [ ]:
predictions = pd.DataFrame({"xgb":xgb_preds, "lasso":lasso_preds})
predictions.plot(x = "xgb", y = "lasso", kind = "scatter")

Many times it makes sense to take a weighted average of uncorrelated results - this usually imporoves the score although in this case it doesn't help that much.

In [ ]:
preds = 

In [ ]:
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("ridge_sol.csv", index = False)